In [ ]:
source("/root/workspace/code/sc-transformer/preprocess/utils.R")
setwd("/root/workspace/code/sc-transformer/")
library(patchwork)


ref_dir <- "data/raw/rna+adt/wnn"
data_dirs <- c(
    "data/raw/atac+rna+adt/tea/TEA-seq/w1",
    "data/raw/atac+rna+adt/tea/TEA-seq/w3",
    "data/raw/atac+rna+adt/tea/TEA-seq/w4",
    "data/raw/atac+rna+adt/tea/TEA-seq/w5",
    "data/raw/atac+rna+adt/tea/TEA-seq/w6",
    "data/raw/atac+rna+adt/dogma/lll_ctrl",
    "data/raw/atac+rna+adt/dogma/lll_stim",
    "data/raw/atac+rna+adt/dogma/dig_ctrl",
    "data/raw/atac+rna+adt/dogma/dig_stim"
)

In [ ]:
# Load reference data
reference <- LoadH5Seurat(pj(ref_dir, "multi.h5seurat"))

In [ ]:
reference <- SCTransform(reference, assay = "ADT", new.assay.name = "SCT", verbose = T)

In [10]:
reference

An object of class Seurat 
21185 features across 161764 samples within 3 assays 
Active assay: SCT_ADT (228 features, 228 variable features)
 2 other assays present: SCT, ADT
 6 dimensional reductions calculated: apca, aumap, pca, spca, umap, wnn.umap

In [11]:
data_dirs <- c(
    # dogma
    "data/raw/atac+rna+adt/dogma/lll_ctrl",
    "data/raw/atac+rna+adt/dogma/lll_stim",
    "data/raw/atac+rna+adt/dogma/dig_ctrl",
    "data/raw/atac+rna+adt/dogma/dig_stim",
    # tea
    "data/raw/atac+rna+adt/tea/TEA-seq/w1",
    "data/raw/atac+rna+adt/tea/TEA-seq/w3",
    "data/raw/atac+rna+adt/tea/TEA-seq/w4",
    "data/raw/atac+rna+adt/tea/TEA-seq/w5",
    "data/raw/atac+rna+adt/tea/TEA-seq/w6",
    # asap
    "data/raw/atac+adt/asap/PBMC_stimulation/asap_ctrl",
    "data/raw/atac+adt/asap/PBMC_stimulation/asap_stim",
    # asap cite
    "data/raw/rna+adt/asap/PBMC_stimulation/cite_ctrl",
    "data/raw/rna+adt/asap/PBMC_stimulation/cite_stim",
    # wnn
    "data/raw/rna+adt/wnn/p1_0",
    "data/raw/rna+adt/wnn/p2_0",
    "data/raw/rna+adt/wnn/p3_0",
    "data/raw/rna+adt/wnn/p4_0",
    "data/raw/rna+adt/wnn/p5_0",
    "data/raw/rna+adt/wnn/p6_0",
    "data/raw/rna+adt/wnn/p7_0",
    "data/raw/rna+adt/wnn/p8_0"
)

In [ ]:
data_dir <- "data/raw/atac+rna+adt/dogma/lll_ctrl"
query_dir <- pj(data_dir, "seurat")
label_dir <- pj(data_dir, "label_seurat_adt")
mkdir(label_dir, remove_old = T)
query <- LoadH5Seurat(pj(query_dir, "adt.h5seurat"))

In [ ]:
query <- SCTransform(query, assay = "adt", new.assay.name = "SCT", verbose = T)

In [23]:
reference <- RunSPCA(reference, assay = "SCT", graph = 'wsnn')

Computing sPCA transformation



In [24]:
anchors <- FindTransferAnchors(
    reference = reference,
    query = query,
    normalization.method = "SCT",
    reference.reduction = "spca",
    # recompute.residuals = F,
    dims = 1:50
)

Normalizing query using reference SCT model



ERROR: Error in umi[residual.features, , drop = FALSE]: invalid or not-yet-implemented 'Matrix' subsetting


In [ ]:
for (data_dir in data_dirs) {
    query_dir <- pj(data_dir, "seurat")
    label_dir <- pj(data_dir, "label_seurat_adt")
    mkdir(label_dir, remove_old = T)

    # Load query data
    query <- LoadH5Seurat(pj(query_dir, "adt.h5seurat")) %>%
             SCTransform(assay = "adt", new.assay.name = "SCT_ADT", verbose = T)
    # SaveH5Seurat(query, pj(query_dir, "rna_sct.h5seurat"), overwrite = T)
    # query <- LoadH5Seurat(pj(query_dir, "rna_sct.h5seurat"))

    # Find mapping
    anchors <- FindTransferAnchors(
        reference = reference,
        query = query,
        normalization.method = "SCT_ADT",
        reference.reduction = "spca",
        # recompute.residuals = F,
        dims = 1:50
    )

    # # Transfer labels
    # query <- MapQuery(
    #     anchorset = anchors,
    #     query = query,
    #     reference = reference,
    #     refdata = list(celltype.l1 = "celltype.l1",
    #                    celltype.l2 = "celltype.l2",
    #                    celltype.l3 = "celltype.l3",
    #                    predicted_ADT = "ADT"),
    #     reference.reduction = "spca",
    #     reduction.model = "wnn.umap"
    # )
    # write.csv(query@meta.data$predicted.celltype.l1, file = pj(label_dir, "l1.csv"))
    # write.csv(query@meta.data$predicted.celltype.l2, file = pj(label_dir, "l2.csv"))
    # write.csv(query@meta.data$predicted.celltype.l3, file = pj(label_dir, "l3.csv"))

    # # Visualize
    # p1 <- DimPlot(query, reduction = "ref.umap", group.by = "predicted.celltype.l1", label = T,
    #          label.size = 3, repel = T) + NoLegend()
    # p2 <- DimPlot(query, reduction = "ref.umap", group.by = "predicted.celltype.l2", label = T,
    #          label.size = 3 ,repel = T) + NoLegend()
    # p3 <- DimPlot(query, reduction = "ref.umap", group.by = "predicted.celltype.l3", label = T,
    #          label.size = 3 ,repel = T) + NoLegend()
    # p1 + p2 + p3
    # ggsave(file = file.path(label_dir, "ref_map.png"), width = 18, height = 6)
}